# NFD Playbook: One node

Simply SSH into the node and use ndn-tool CMD commands.

For example, chunks.

On Terminal 1 (node 1):
* dd if=/dev/zero of=file.txt bs=1GB count=2
* ndnputchunks /general/data/4319 < file.txt

On Terminal 2 (node 2):
* ndncatchunks /general/data/4319 > output.txt

## Preliminaries

### Imports

In [ ]:
from fabrictestbed_extensions.fablib.fablib import fablib

### Parameters and Variables

In [ ]:
slice_name = 'NFD_1Node'
sites = ['SALT']
nodes_name = ['Node1']

image = 'default_ubuntu_20'
cores = 48
ram = 128
disk = 100

go_version = '1.20.4'

### List available resources by site

In [ ]:
try:
    fablib.list_sites()
except Exception as e:
    print(f"Exception: {e}")

## Create slice

### Submit slice

In [ ]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)

    # Node1
    node1 = slice.add_node(name=nodes_name[0], site=sites[0])
    node1.set_capacities(cores=cores, ram=ram, disk=disk)
    node1.set_image(image)
    
    #Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Slice Fail: {e}")

### Get slice

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

### Get all nodes

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)

    print(f"{slice.list_nodes()}")
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print(f"{node}")
except Exception as e:
    print(f"Exception: {e}")

### Install dependencies

In [ ]:
%%capture
try:
    slice.get_nodes()[0].execute(f'''
        echo "deb [arch=amd64 trusted=yes] https://nfd-nightly-apt.ndn.today/ubuntu focal main" | sudo tee /etc/apt/sources.list.d/nfd-nightly.list
        sudo DEBIAN_FRONTEND=noninteractive apt update
        sudo DEBIAN_FRONTEND=noninteractive apt install -y libndn-cxx-dev nfd ndnping ndnpeek ndn-dissect ndnchunks ndnsec
        sudo DEBIAN_FRONTEND=noninteractive apt full-upgrade -y
        sudo reboot
    ''')
    slice.wait_ssh(progress=True)
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
%%capture
try:
    slice.get_nodes()[0].execute(f'''
        curl -O -L "https://golang.org/dl/go{go_version}.linux-amd64.tar.gz"
        sudo tar -xf "go{go_version}.linux-amd64.tar.gz" -C /usr/local
        rm "go{go_version}.linux-amd64.tar.gz"
        echo "export GOPATH=$HOME/go" >> ~/.bashrc
        echo "export PATH=$PATH:/usr/local/go/bin:$GOPATH/bin" >> ~/.bashrc
        source ~/.bashrc
    ''')
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
try:
    slice.get_nodes()[0].execute(f'''
        nfd-stop
    ''')
    slice.get_nodes()[0].upload_file('./nfd.conf', 'nfd.conf')
    slice.get_nodes()[0].execute(f'''
        chmod 644 nfd.conf
        sudo mv nfd.conf /etc/ndn/nfd.conf
        nfd-start
        ndnsec key-gen /ndn/fabric/node1 | ndnsec cert-install -
    ''')
except Exception as e:
    print(f"Exception: {e}")

## Renew slice

In [ ]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

days_more = 5
end_date = (datetime.now(timezone.utc) + timedelta(days=days_more)).strftime("%Y-%m-%d %H:%M:%S %z")

try:
    slice = fablib.get_slice(name=slice_name)
    slice.renew(end_date)
except Exception as e:
    print(f"Exception: {e}")

## Delete slice

In [ ]:
try:
    slice = fablib.delete_slice(slice_name)
except Exception as e:
    print(f"Exception: {e}")